<a href="https://colab.research.google.com/github/sam1610/Inference_DIRinfo/blob/master/30_7_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment prepartion & libraries installation

 Links

- https://dash.plotly.com/basic-callbacks
- https://beta.packtpub.com/book/data/9781800568532/6/ch06lvl1sec27/controlling-seasonality-with-fourier-order

- https://github.com/sam1610/Inference_DIRinfo

In [1]:
! pip  install -U   scikit-learn  random_timestamp ipywidgets plotly

In [2]:
import pandas as pd  
import numpy as np
import random
import string
from random import randint
import matplotlib.pyplot as plt
from datetime import timedelta
import seaborn as sns
import plotly.graph_objects as go

In [3]:
import ipywidgets as widgets 
from ipywidgets import interact
from matplotlib.dates import DateFormatter
from datetime import timedelta
from datetime import datetime
from ipywidgets import interact, Dropdown , Layout , Box , Button
from ipywidgets import Checkbox
from plotly.subplots import make_subplots

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!ls  /content/drive/MyDrive/portalAnalysis/

 Data	     output_filename.html	   'سؤال خاطئ3.pdf'
 dist	     positives.html		   'سؤال صحيح1.pdf'
 link.html   TimeSeriesAnalysisWithPython
 mwe.pdf    'سؤال خاطئ2.pdf'


In [7]:
cd /content/drive/MyDrive/portalAnalysis/

/content/drive/MyDrive/portalAnalysis


#Loading data sets

## Student's report dataset

In [8]:
dd=pd.read_csv("./Data/student_report.csv", index_col=0)

In [9]:
dd["Activity ID"].replace({"Act_14": "اختبار_14", "Act_12": "اختبار_12","Act_2":"نشاط_2" , "Act_4":"نشاط_4" ,
                           "Act_16":"اختبار_16","Act_3":"نشاط_3" ,"Act_15":"اختبار_15" ,"Act_13":"اختبار_13" ,"Act_11":"اختبار_11" ,
                           "Act_18":"اختبار_18","Act_10":"اختبار_10" ,"Act_1":"نشاط_1" ,"Act_17":"اختبار_17"}, inplace=True)

In [10]:
dd=dd.reset_index()

In [11]:
dd=dd.drop(['Activity Type'], axis = 1)

In [12]:
dd

,Submit_Date,Activity ID,Mark
0,31-03-2020,اختبار_14,2.31
1,01-04-2020,اختبار_12,0.61
2,07-05-2020,نشاط_2,0.80
3,10-05-2020,نشاط_4,2.23
4,26-06-2020,اختبار_16,1.64
5,07-07-2020,نشاط_3,2.12
6,03-08-2020,اختبار_15,4.20
7,17-08-2020,اختبار_13,7.25
8,23-08-2020,اختبار_11,9.73
9,06-09-2020,اختبار_18,5.37


In [13]:
dd.columns = ['تاريخ التسليم', 'اسم التقييم',"الدرجة"]

## Load General-Students-Info.csv   (( df_st ))

In [14]:
df_st=pd.read_csv("./Data/General_students_info.csv", index_col=0,converters={'student_ID': lambda x: str(x) , # lambda used to keep zeros to the left of no.s
                                                                              'birth_year': lambda x: str(x) ,
                                                                            'birth_month': lambda x: str(x)})

In [15]:
df_st.head()

,student_ID,birth_year,birth_month,school,grade,gender,governorate
0,100315215,10,03,Sc_6,5,M,Caiptal
1,070403058,07,04,Sc_8,8,F,North
2,080748524,08,07,Sc_4,7,M,Muharraq
3,071090450,07,10,Sc_9,8,F,Caiptal
4,090636469,09,06,Sc_6,6,M,Caiptal


## Load Student's Portal Achievments (( df_avt ))

In [16]:
df_avt=pd.read_csv("./Data/Portal-Achiev-Info.csv", index_col=0,converters={'student_ID': lambda x: str(x)})

In [17]:
df_avt.head()

,student_ID,Course_code,Activity_id,Activity_type,Unit_id,Chapter_id,Lesson_id,Act_diff,Act_cp_count,Act_time,Act_submit_Time,Act_mark
0,080748524,g7_1,g7_1-Act_12,1,Un_6,Chp_7,Ls_1,1,2,884,2020-06-06 08:07:04,5.74
1,080748524,g7_2,g7_2-Act_1,1,Un_4,Chp_9,Ls_0,3,1,3098,2020-01-09 10:09:03,7.02
2,080748524,g7_1,g7_1-Act_18,0,Un_4,Chp_2,Ls_5,4,2,15,2020-12-20 17:35:27,7.67
3,080748524,g7_1,g7_1-Act_15,1,Un_1,Chp_0,Ls_2,4,2,1458,2020-12-21 15:11:32,6.22
4,080748524,g7_2,g7_2-Act_14,0,Un_5,Chp_2,Ls_7,4,2,14,2020-04-27 22:10:18,8.96


## Load the dataset that has 5 selected students Info (( dt_st ))

In [18]:
dt_st =pd.read_csv("./Data/5_selected_students.csv", converters={'student_ID': lambda x: str(x)})

In [19]:
dt_st["Act_submit_Time"]= pd.to_datetime(dt_st.Act_submit_Time)

In [20]:
dt_st[dt_st.Activity_type=='0']["Act_time"]=dt_st.query("Activity_type=='0'").Act_time/60

In [21]:
dt_st.set_index("Act_submit_Time", inplace=True)

In [22]:
dt_st.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 195 entries, 2020-10-07 20:44:56 to 2020-04-01 04:25:24
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   student_ID     195 non-null    object 
 1   Course_code    195 non-null    object 
 2   Activity_id    195 non-null    object 
 3   Activity_type  195 non-null    int64  
 4   Unit_id        195 non-null    object 
 5   Chapter_id     195 non-null    object 
 6   Lesson_id      195 non-null    object 
 7   Act_diff       195 non-null    int64  
 8   Act_cp_count   195 non-null    int64  
 9   Act_time       195 non-null    int64  
 10  Act_mark       195 non-null    float64
dtypes: float64(1), int64(4), object(6)
memory usage: 18.3+ KB


## Load the dataset that has statistics about students achievments (( df_mg ))

In [23]:
df_mg=pd.read_csv("./Data/student_statistics.csv", index_col=0,converters={'student_ID': lambda x: str(x)})

In [24]:
df_mg.head()

,student_ID,Course_code,Activity_type,Std_act_cp_mean,Std_act_d_mean,Std_act_m_count,Std_act_m_max,Std_act_m_mean,Std_act_m_std,Std_act_t_max,Std_act_t_mean,Std_act_t_std,Act_score,Act_cp_count_mean,Act_diff_mean,Act_mark_count,Act_mark_max,Act_mark_mean,Act_mark_min,Act_time_max,Act_time_mean,Act_time_min,Activities_Count_<lambda>,last_mark
0,060130518,g9_1,0,2.000000,2.0,1,3.57,3.570000,0.000000,15,15.000000,0.000000,4.996892,1.4125,2.4375,80,9.82,5.3555,0.02,16.0,9.35,2.0,20,3.57
1,060131582,g9_1,0,1.000000,2.0,2,9.82,5.705000,5.819489,7,6.000000,1.414214,20.525019,1.4125,2.4375,80,9.82,5.3555,0.02,16.0,9.35,2.0,20,9.82
2,060149111,g9_1,0,1.000000,4.0,1,8.52,8.520000,0.000000,13,13.000000,0.000000,13.662241,1.4125,2.4375,80,9.82,5.3555,0.02,16.0,9.35,2.0,20,8.52
3,060155495,g9_1,0,1.000000,3.5,2,5.90,4.755000,1.619275,8,7.500000,0.707107,10.384306,1.4125,2.4375,80,9.82,5.3555,0.02,16.0,9.35,2.0,20,3.61
4,060161018,g9_1,0,1.666667,1.0,3,7.94,5.103333,3.090410,8,5.666667,2.081666,14.520127,1.4125,2.4375,80,9.82,5.3555,0.02,16.0,9.35,2.0,20,5.56


## Load  Student Classification dataset 

In [25]:
std_classPred=pd.read_csv("./Data/std_classPred.csv", index_col=[0, 1,2],converters={'student_ID': lambda x: str(x)})
std_classPred.head()

Predicted_class
student_ID Course_code Activity_type                
080748524S g5_1        0                           A
                       1                           A
           g5_2        0                           A
                       1                           C
           g5_3        0                           A

## Load the predicted data set (( predict_std ))

In [26]:
predict_std=pd.read_csv("./Data/predicted_data.csv", index_col=0,converters={'student_ID': lambda x: str(x)})

In [27]:
predict_std.head()

,Course_code,Activity_type,Std_act_cp_mean,Std_act_d_mean,Std_act_m_count,Std_act_m_max,Std_act_m_mean,Std_act_m_std,Std_act_t_max,Std_act_t_mean,Std_act_t_std,Act_score,Act_cp_count_mean,Act_diff_mean,Act_mark_count,Act_mark_max,Act_mark_mean,Act_mark_min,Act_time_max,Act_time_mean,Act_time_min,Activities_Count_<lambda>,last_mark
student_ID,,,,,,,,,,,,,,,,,,,,,,,
080748524S,g5_1,0,1.777778,2.333333,9,9.35,4.958889,3.068291,16,8.888889,3.480102,34.174257,1.474747,2.616162,99,9.85,4.810000,0.01,16.0,8.797980,2.0,20,5.083149
080748524S,g5_1,1,1.750000,3.750000,4,9.46,8.155000,1.558001,4285,2741.750000,1564.793570,24.263784,1.494845,2.567010,97,9.83,5.157732,0.06,4356.0,2179.505155,91.0,20,7.524903
080748524S,g5_2,0,1.222222,2.777778,9,9.30,4.138889,3.379750,16,10.444444,5.270463,37.186354,1.456790,2.407407,81,9.83,5.190864,0.00,16.0,8.703704,2.0,19,4.476848
080748524S,g5_2,1,1.500000,3.000000,4,5.66,4.780000,0.914440,4130,2854.750000,1300.834444,12.990291,1.445545,2.445545,101,9.70,5.100099,0.08,4373.0,2444.118812,102.0,20,4.802190
080748524S,g5_3,0,1.555556,2.555556,9,9.73,4.420000,2.838596,16,9.666667,4.609772,40.518231,1.449438,2.359551,89,9.99,4.815506,0.04,16.0,8.842697,2.0,20,4.991331


## Load Recommend data set (( df_recommend ))

In [30]:
df_recommend=pd.read_csv("./Data/recommend.csv", index_col=0,converters={'student_ID': lambda x: str(x)})

In [31]:
df_recommend.head()

,student_ID,Lesson_des,lobj_id,lobj_des,LearningObj
0,080748524S,Mean and median,7,"Mean, Median & Mode example","Mean and median Mean, Median & Mode example"
1,080748524S,Dot plots & frequency tables,1,Representing Data,Dot plots & frequency tables Representing Data
2,080748524S,Interquartile range (IQR),14,Interquartile range (IQR),Interquartile range (IQR) Interquartile range ...
3,080748524S,Dot plots & frequency tables,2,Frequency Tables & dot plot,Dot plots & frequency tables Frequency Tables ...
4,080748524S,Mean absolute deviation (MAD),21,Mean absolute deviation example,Mean absolute deviation (MAD) Mean absolute de...


In [32]:
#  Json  recommendation Initialization
modules=[
         {"unit":"Data and statistics", 
          "lessons":[
                    {"LessName": "Dot plots & frequency tables",
                    "lobs":[
                            {"lobID": 1, "lobName":"Representing Data", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/dot-plot/v/ways-to-represent-data?modal=1" },
                            {"lobID": 2,"lobName":"Frequency Tables & dot plot", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/dot-plot/v/frequency-tables-and-dot-plots?modal=1"}
                          ]
                          
                    },
                    {"LessName": "Statistical questions",
                    "lobs":[
                            {"lobID": 3, "lobName":"Statistical questions", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-statistical-questions/v/understanding-statistical-questions?modal=1"}
                          ]
                    },
                    {"LessName": "Histograms",
                    "lobs":[
                            {"lobID": 4, "lobName":"Creating a histogram", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/histograms/v/histograms-intro?modal=1"},
                            {"lobID": 5, "lobName":"Interpreting a histogram", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/histograms/v/interpreting-histograms?modal=1"}
                          ]
                    },
                    {"LessName": "Mean and median",
                    "lobs":[
                            
                                {"lobID": 6, "lobName":"Statistics Intro: Mean, Median & Mode", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/mean-and-median/v/statistics-intro-mean-median-and-mode?modal=1"}, 
                                {"lobID": 7, "lobName":"Mean, Median & Mode example", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/mean-and-median/v/mean-median-and-mode?modal=1"},
                                {"lobID": 8 , "lobName":"Calculating the mean", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/mean-and-median/a/calculating-the-mean?modal=1"}
                                
                                
                          ]
                    },
                    {"LessName": "Mean and median challenge problems",
                    "lobs":[
                             {"lobID": 9, "lobName": "Missing value given the mean", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/using-mean-to-find-missing-value?modal=1"},
                              {"lobID":10 , "lobName":"Mean as the balancing point" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/a/mean-as-the-balancing-point?modal=1"},
                              {"lobID": 11, "lobName":"Impact on median & mean: Removing an outlier" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/impact-on-median-and-mean-when-removing-lowest-value-example?modal=1"},
                              {"lobID": 12, "lobName":"Impact on median & mean: increasing an outlier" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/impact-on-median-and-mean-when-increasing-highest-value?modal=1"},
                              {"lobID": 13, "lobName":"Median & range puzzlers" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/median-and-range-puzzle?modal=1"}
                          ]
                    },
                    {"LessName": "Interquartile range (IQR)",
                    "lobs":[
                            {"lobID": 14, "lobName":"Interquartile range (IQR)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th/v/calculating-interquartile-range-iqr?modal=1"}
                          ]
                    },
                    {"LessName": "Box Plots",
                    "lobs":[
                            {"lobID":15  , "lobName":"Reading box plots", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/reading-box-and-whisker-plots?modal=1"},
                             {"lobID":16  , "lobName":"Constructing a box plot", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/constructing-a-box-and-whisker-plot?modal=1"},
                             {"lobID":17  , "lobName":"Worked example: Creating box plot (odd number of data points)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/box-and-whisker-plot-exercise-example?modal=1"},
                             {"lobID":18  , "lobName":"Worked example: Creating box plot (even number of data points)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/constructing-a-box-and-whisker-plot?modal=1"},
                             {"lobID":19  , "lobName":"Worked example: Interpreting a box plot", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/interpreting-box-plots?modal=1"}
                          ]
                    },

                    {"LessName": "Mean absolute deviation (MAD)",
                    "lobs":[
                            { "lobID":20 , "lobName":"Mean absolute deviation (MAD)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-mad/v/mean-absolute-deviation?modal=1"},
                              {"lobID":21 , "lobName":"Mean absolute deviation example", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-mad/v/mean-absolute-deviation-example?modal=1"}
                          ]
                    },
                    {"LessName": "Comparing Data Display",
                    "lobs":[
                            {"lobID":22 , "lobName":"Comparing dot plots, histograms and box plots", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-7th-compare-data-displays/v/comparing-dot-plots-histograms-and-box-plots?modal=1"}
                          ]
                    },
                    {"LessName": "Shape of Data Distribution",
                    "lobs":[
                            {"lobID":23 , "lobName":"Shape of distributions", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-shape-of-data/v/shapes-of-distributions?modal=1"},
                              {"lobID":24 , "lobName":"Clusters, gaps, peaks & outliers", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-shape-of-data/v/examples-analyzing-clusters-gaps-peaks-and-outliers-for-distributions?modal=1"}
                          ]
                    },
                    

                    
                    ],
           
          
          }
         ]

# Graph the selected student's marks , predictions and recommendations


In [33]:
import ipywidgets as widgets 
from ipywidgets import interact
from matplotlib.dates import DateFormatter
from datetime import timedelta

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
std_d={}
for st in  dt_st.student_ID.unique():
  std_d[st]=dt_st[dt_st.student_ID==st]["Course_code"].unique()

In [36]:
from ipywidgets import interact, Dropdown, Checkbox, DatePicker
from IPython.core.display import display, HTML
from datetime import date


stID1 = Dropdown(options = std_d.keys())
stID2 = Dropdown(options = std_d.keys())
stID3 = Dropdown(options = std_d.keys())
stID4 = Dropdown(options = std_d.keys())
stID5 = Dropdown(options = std_d.keys())
stID6 = Dropdown(options = std_d.keys())



course1 = Dropdown(options = std_d[stID1.value]) # options = std_d[stID.value] is to remove inital error but not that necessary.
course2 = Dropdown(options = std_d[stID2.value]) # options = std_d[stID.value] is to remove inital error but not that necessary.
course3 = Dropdown(options = std_d[stID3.value]) # options = std_d[stID.value] is to remove inital error but not that necessary.
course4 = Dropdown(options = std_d[stID4.value]) # options = std_d[stID.value] is to remove inital error but not that necessary.
course5 = Dropdown(options = std_d[stID5.value]) # options = std_d[stID.value] is to remove inital error but not that necessary.
course6 = Dropdown(options = std_d[stID6.value]) # options = std_d[stID.value] is to remove inital error but not that necessary.




D1=DatePicker(
    description='ابتداءًا من تاريخ')
D2=DatePicker(
    description='ابتداءًا من تاريخ')
D3=DatePicker(
    description='ابتداءًا من تاريخ')
D4=DatePicker(
    description='ابتداءًا من تاريخ')    
D5=DatePicker(
    description='ابتداءًا من تاريخ')
D6=DatePicker(
    description='ابتداءًا من تاريخ')



D1.value=pd.to_datetime("2020-01-01")
D2.value=pd.to_datetime("2020-01-01")
D3.value=pd.to_datetime("2020-01-01")
D4.value=pd.to_datetime("2020-01-01")
D5.value=pd.to_datetime("2020-01-01")
D6.value=pd.to_datetime("2020-01-01")


box1 = Checkbox(False, description='عرض الأنشطة')
box2 = Checkbox(False, description='عرض الأنشطة')
box3 = Checkbox(False, description='عرض الأنشطة')
box4 = Checkbox(False, description='عرض الأنشطة')
box5 = Checkbox(False, description='عرض الأنشطة')
box6 = Checkbox(False, description='عرض الأنشطة')



# @interact(st = stID1, cs = course1, dt=D1, chkbx= box1)
# def print_cs(st, cs, dt, chkbx ):
#     course1.options = std_d[st] # Here is the trick, i.e. update course.options based on st, namely stID.value.
#     # print(st, cs)
#     box1.description = " Assignement"   if chkbx   else "Quizz"



In [37]:
def  recommend(st):
#  full code for recommendation 
    

# Current Student Actual. learningobjects
  df_st_Rec=df_recommend.query(f"student_ID=='{st}'").drop_duplicates(subset=["lobj_id"], keep="first")
##----Generate Vecrorized DF--------------------
  sorted_score=[]
  lst_obj=[]

  
  for lobInd in list(df_st_Rec.index):
    df_vectorize= df_recommend[df_recommend.student_ID !=st].append(
    {"student_ID":st, "Lesson_des":df_st_Rec.loc[lobInd,"Lesson_des" ],"lobj_id":df_st_Rec.loc[lobInd,"lobj_id"],
     "lobj_des":df_st_Rec.loc[lobInd,"lobj_des"],"LearningObj":df_st_Rec.loc[lobInd,"LearningObj"] }, ignore_index=True)  
    cs= cosine_similarity(CountVectorizer().fit_transform(df_vectorize.LearningObj))
    sorted_score += sorted(list(enumerate(cs[df_vectorize.shape[0]-1])), key=lambda x: x[1], reverse=True)
    sorted_score.sort( key=lambda x : x[1] , reverse=True)

  for  rec in sorted_score:
    lst_obj.append(df_vectorize.loc[rec[0], "lobj_des"])
    lst_obj=list(dict.fromkeys(lst_obj))
        # take  first  5 elements
    lst_obj=[  lob   for  lob in  lst_obj  if lob not in df_st_Rec.lobj_des.unique()]

  return lst_obj
  #return recommend

In [38]:
recommend("080748524S")

['Constructing a box plot',
 'Mean as the balancing point',
 'Missing value given the mean',
 'Worked example: Interpreting a box plot',
 'Worked example: Creating box plot (even number of data points)',
 'Worked example: Creating box plot (odd number of data points)',
 'Interpreting a histogram',
 'Creating a histogram',
 'Statistical questions']

In [39]:
def url_lobj(lob):
  
  for i in modules:
    for j in i["lessons"]:
      for k in j["lobs"]:
        if k["lobName"]==lob:
          return  k["URL"]


In [40]:
def plot_std_Data1(st, cs, dt, chk):
  #fig,ax=plt.subplots( figsize=(20,10) )

  dt_plot=dt_st.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}' & Activity_type=='{chk}' & Act_submit_Time>='{dt}' ")
  if (dt_plot.shape[0]>0):
    dt_plot.set_index("Act_submit_Time", inplace=True, drop=True)
    fig,ax=plt.subplots( figsize=(20,10) )

    course1.options = std_d[st]
    plt.rcParams.update({'font.size': 25})
    
    print(list(std_classPred.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}' & Activity_type=='{chk}' ").Predicted_class)[0])
    ax1=ax.twinx()

    lst_lobj=recommend(st)
    plt.subplots_adjust(top = 0.99, bottom=0.01,hspace=0.4)

       
    ind_xticks=dt_st.query(f"student_ID=='{st}' & Course_code=='{cs}' & Activity_type=='{chk}' ").index

    ref_q=dt_st.reset_index().query(f"student_ID=='{st}' & Course_code=='{cs}' & Activity_type=='{chk}' ").Act_submit_Time.max()
    x_point=[ref_q, ref_q+ timedelta(days=7)]
    y_point=[dt_st.loc[ref_q, "Act_mark"], predict_std.reset_index().query(f"student_ID=='{st}'  & Course_code=='{cs}' & Activity_type=='{chk}' ").last_mark]
    ax.plot(x_point , y_point, linestyle='dashed', marker='o',
                  markerfacecolor='blue', markersize=12)
        

    #------- ADD TEXT TO THE GRAPH ------------------
           
    fig.text(0.45,-0.6, "You may like to watch these recommendations:", size=22, ha="center", color="purple",horizontalalignment='right',
            bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'),
            transform=ax1.transAxes)
    fig.text(0.5,-0.33," " ) #### space below the above sentence

  
  
    ### (so) is the value of predicted mark 
    so=str("{:.2f}".format(list(y_point[1])[0]))
    fig.text(0.32,1.2, "Your predicted Mark:", size=25, ha="center", color="blue",horizontalalignment='left',
            bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'),transform=ax1.transAxes)
    fig.text(0.6,  1.15 ,  so ,color="blue" , size=25 ,bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'))### predicted mark
    fig.text(0.65,1.21," " )#### space above the above sentence
      #-----------------------------------------------------------
    dt_plot.Act_mark.plot( 
        subplots=True,marker="o",    ax=ax , color="brown")
    
    dt_plot.Act_time.plot( 
        subplots=True, marker="s",   ax=ax1 , color="green")

    Alabel="Assignment marks"   if chk  else  "Quizzes marks"
    Atitle= "Assignmenrs marks / month"  if chk   else "Quizzes marks / month"

    # ---------------------------------------------------------------------
    ax.set(ylabel= Alabel , xlabel = "Dates") 
    
    L=ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left',)
    L.get_texts()[0].set_text('Activity marks')

    Qlabel="Assignment submit time / minutes"   if chk  else  "Quizzes submit time / minutes"
    T_title= "Assignments marks compared to submit time"  if chk   else "Quizzes marks compared to submit time"

    ax1.set(
          ylabel= Qlabel, 
          title=T_title
          )
    ax1.grid()
    B=ax1.legend(bbox_to_anchor=(1.05, 1), loc='lower left')
    B.get_texts()[0].set_text('Submit time')
    fig.autofmt_xdate(rotation=75)
    plt.show()


    #### printing URLs for recommendatios 

    for item in recommend(st):
      display(HTML(f"<a href={url_lobj(item)} target='_blank' style='font-size:1.3em'>{item}</a>"))
    
  else:
    print(" No records Available")
      # for item in recommend(st):
    

In [41]:
@interact(st = stID1, cs = course1, dt = D1, chkbx = box1)
def plot_data( st, cs, dt , chkbx):
  #print(((chkbx)+0))
  box1.description = " Assignement"   if chkbx   else "Quizz"
  course1.options = std_d[st]

  # print(predict_std.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}'"))
  plot_std_Data1(st, cs, dt , ((chkbx)+0))



interactive(children=(Dropdown(description='st', options=('080748524S', '090904691S', '060991839S', '071059051…

## Necessary Recommendations



In [42]:
target_st = dt_st[["student_ID",	"Course_code",	"Activity_id"]]

In [43]:
target_st.drop_duplicates(subset=["Course_code"], keep="last", inplace=True)

In [44]:
target_st.sample(10)

,student_ID,Course_code,Activity_id
Act_submit_Time,,,
2020-10-20 09:53:42,060991839S,g6_1,g6_1-Act_4
2020-10-07 23:17:23,090904691S,g9_2,g9_2-Act_4
2020-10-19 19:50:48,060991839S,g6_2,g6_2-Act_4
2020-12-20 20:41:44,071223506S,g7_1,g7_1-Act_4
2020-09-21 11:23:53,071223506S,g7_3,g7_3-Act_4
2020-02-26 22:15:50,080748524S,g5_2,g5_2-Act_4
2020-10-11 22:14:50,071059051S,g8_3,g8_3-Act_4
2020-04-01 04:25:24,071223506S,g7_2,g7_2-Act_4
2020-11-22 03:18:07,060991839S,g6_3,g6_3-Act_4


In [45]:
def rec_repo( s,t, c, a, q):
  return {
          "student_ID": s,
          "Act_submit_Time": t ,
          "Course_code":c,  
          "Activity_id": a,
          "Q_id": q,
          "Q_mark":random.choices( np.arange(1, 4), weights=[30,60,40])[0], 
          "PF": random.choices([1, 0], weights=(60, 40))[0]
      }

In [46]:
def Q_repo( q, lob):
  return {
          "Q_id": q,
          "lob_id":lob 
      }

In [47]:
def  std_recd( lst):
  lstUrl=[]
  for i in modules:
    for j in i["lessons"]:
      for k in j["lobs"]:
        if k["lobID"] in lst:
          lstUrl.append((k["URL"], k["lobName"]))
  return lstUrl


In [48]:
lst_rec=[]
lst_Q=[]
#std_array= dt_st.student_ID
#std_course= dt_st.query("student_ID=='080748524S'").Course_code.unique()
#std_act= dt_st.query("student_ID=='080748524S'").Activity_id.unique()

for  s,t, course , act in zip(target_st.student_ID,target_st.index, target_st.Course_code , target_st.Activity_id):
  for Q in [f"Q{act}_{rd}" for rd in range(int(9*np.random.rand()+2))]:
    lst_rec.append(rec_repo(s,t, course, act, Q))
    for lob  in random.choices(np.arange(24), k=int(6*np.random.rand()+4)):
      lst_Q.append(Q_repo(Q, lob))
               
df_Rrepo=pd.DataFrame(lst_rec)
df_Qrepo=pd.DataFrame(lst_Q)


In [49]:
%%time
df_Rrepo.sample(1)

CPU times: user 533 µs, sys: 57 µs, total: 590 µs
Wall time: 599 µs


,student_ID,Act_submit_Time,Course_code,Activity_id,Q_id,Q_mark,PF
25,090904691S,2020-02-28 04:14:03,g9_3,g9_3-Act_4,Qg9_3-Act_4_4,2,0


In [50]:
std_new={}
for st in  target_st.student_ID.unique():
  std_new[st]=target_st[target_st.student_ID==st]["Course_code"].unique()

In [51]:
stID7 = Dropdown(options = std_new.keys())
course7 = Dropdown(options = std_new[stID7.value])


##################
#course4 = Dropdown(options = std_d["080748524S"]) # options = std_d[stID.value] is to remove inital error but not that necessary.
D7=DatePicker(
    description='Starting date')
D7.value=pd.to_datetime("2020-01-01")

In [52]:
def  plot_strec( st,cs,dt ):
  lstrec=[]
  
  _df_Rrepo= df_Rrepo.query(f"student_ID=='{st}' & Course_code=='{cs}' & Act_submit_Time>='{dt}'")
  
  if _df_Rrepo.shape[0]>0:
    
    fig_dims = (20, 8)
    fig, ax = plt.subplots(figsize=fig_dims)
    plt.rcParams.update({'font.size': 25})


    g=sns.barplot( 
        x="Q_id" ,y= "Q_mark", 
        hue="PF", data=_df_Rrepo, ci=None, palette=["C3", "C0" ], ax=ax )
    g.set_xticklabels([f"Q_{m+1}"  for m in range(_df_Rrepo.shape[0])])


        ######## graph formatting #######

    ax.set_title('Passed / Failed answers in the selected activity')
    ax.set(xlabel='Question number', ylabel='Points per question')
    N=plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    N.get_texts()[0].set_text('Wrong Answers')
    if len(N.get_texts())>1:
      N.get_texts()[1].set_text('Correct Answers')

        ###### Text below the graph ######

    fig.text(0.43,-0.6, "Watch these recommendations to learn more:", size=22, ha="center", color="purple",horizontalalignment='right',
            bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'),
            transform=ax.transAxes)
    fig.text(0.5,-0.4," " ) #### space below the above sentence
    fig.text(0.65,1.1," " )#### space above the above sentence

        # plt.show()
      ###########

    for q in _df_Rrepo.query(f"PF=='0'").Q_id.unique():
      lstrec +=list(df_Qrepo.query(f"Q_id=='{q}'").lob_id)
    lstrec=list(dict.fromkeys(lstrec))
    # print(list(dict.fromkeys(lstrec)))
    plt.show()
    
    for item in std_recd(lstrec[:5]):
      display(HTML(f"<a href={item[0] } target='_blank' style='font-size:1.3em'>{item[1]}</a>"))
    
    

  else:
    print(" No Records Available")


In [53]:
%%time
@interact(st=stID7, cs = course7 , dt=D7)
def plot_rec(st, cs, dt):
  course7.options = std_new[st]
  plot_strec( st,cs,  dt)
  

interactive(children=(Dropdown(description='st', options=('080748524S', '090904691S', '060991839S', '071059051…

CPU times: user 516 ms, sys: 38.1 ms, total: 554 ms
Wall time: 553 ms


In [54]:
import plotly.express as px


In [55]:
def plot_Q( st,cs,dt ):
 
  lrec=[]
  
  _df_Rrepo= df_Rrepo.query(f"student_ID=='{st}' & Course_code=='{cs}' & Act_submit_Time>='{dt}' ")
  
  if _df_Rrepo.shape[0]>0:
    
    course2.options = std_new[st]

    

  ####   1
    # fig = go.Figure(go.Bar(
    #             x=_df_Rrepo.Q_id,
    #             y=_df_Rrepo.Q_mark,
    #             width=0.2,
    #             marker=dict(color=np.where(_df_Rrepo["PF"]==0, 'red', 'green'))
    #              ))


  ####     2  
    # fig = go.Figure()
    # fig.add_trace(go.Bar(x=_df_Rrepo.Q_id,y=_df_Rrepo.Q_mark,width=0.2,showlegend=True,marker=dict(color=np.where(_df_Rrepo["PF"]==0, 'red', 'green'))
    # ))


    ####    3

    fig = px.bar(_df_Rrepo, x=_df_Rrepo.Q_id, y=_df_Rrepo.Q_mark,
                        color=_df_Rrepo.PF,
                        color_discrete_map={'0': 'red','1': 'green'},)
        
    
    
    
    
    # var = _df_Rrepo["PF"].unique()
    # fig = go.Figure()
    # for v in var:
    #   fig.add_trace(go.Bar(x=_df_Rrepo.Q_id,
    #                        y=_df_Rrepo.Q_mark,
    #                        width=0.2,
    #                        showlegend=True,
    #                        marker=dict(color=v)
    #                           )
    #              )
               
    
    

    # for idx in range(len(fig.data)):
    #  fig.data[idx].x = [f"Q_{m+1}"  for m in range(_df_Rrepo.shape[0])]


    # fig.update_layout(
    # title_text="Questions from all activities submitted" )
    # fig.update_xaxes(title_text="Questions")
    # fig.update_yaxes(title_text="Point per qustion")
    # fig.update_layout(hovermode="x")
    
    ################ PRINT RECOMM.

    for q in _df_Rrepo.query(f"PF=='0'").Q_id.unique():
      lrec +=list(df_Qrepo.query(f"Q_id=='{q}'").lob_id)
      lrec=list(dict.fromkeys(lrec))
      # print(list(dict.fromkeys(lrec)))
      #fig.show()
      for item in std_recd(lrec[:5]):
        display(HTML(f"<a href={item[0] } target='_blank' style='font-size:1.3em'>{item[1]}</a>"))
        
    fig.show()    
  else:
    print(" No records Available")
      

In [56]:
@interact(st=stID2, cs = course2 , dt=D2)
def plot_QQ(st, cs, dt):
  course2.options = std_new[st]
  plot_Q( st,cs,  dt)

interactive(children=(Dropdown(description='st', options=('080748524S', '090904691S', '060991839S', '071059051…

# Graph using Box plot

## Plot mark VS submit_time ( with box plot )

In [57]:
#Combine Graph

def plot_std_Data2(st, cs, dt ,chk):
  import seaborn as sns 
  import matplotlib.pyplot as plt


  dt_tmp= dt_st.query(
      f"student_ID=='{st}' & Course_code=='{cs}' & Activity_type=='{chk}'"
      )[["Act_mark", "Activity_id" ]]
  dt_tmp.sort_index( inplace=True )
  pd.to_datetime(dt_tmp.index).strftime('%d-%m-%Y')
 
 
  if (dt_tmp.reset_index().query(f"Act_submit_Time>='{dt}'").shape[0]>0):


    fg, ax= plt.subplots( figsize=(20,8))
    course3.options = std_d[st]


  ######### USED TO PICK DATE FROM CALENDER  #########
    # dt_tmp= dt_st.query(
    #     f"student_ID=='{st}' & Course_code=='{cs}'& Activity_type=='{chk}'")[["Act_mark", "Activity_id" ]]
    # dt_tmp=dt_tmp.query(f"Act_submit_Time >= '{dt}'")
    # if dt_tmp.shape[0]>0:
    #   course.options = std_d[st]


    
  ######## Predicted value  ###########
    #   we suppose that  predict_std is already available
    ref_q=dt_st.sort_index().query(
        f"student_ID=='{st}' & Course_code=='{cs}' & Activity_type=='{chk}'"
        ).tail(1).index+timedelta(days=7)
    
    yq_point=[dt_st.sort_index().query(
        f"student_ID=='{st}' & Course_code=='{cs}' & Activity_type=='{chk}' "
        ).Act_mark.tail(1) 
    , predict_std.reset_index().query(
        f"student_ID=='{st}'  & Course_code=='{cs}' & Activity_type=='{chk}' "
        ).last_mark]
    #print(yq_point)
    print(f"  Predicted Mark: {list(yq_point[1])[0]}")

  ##########################################
    df_stAct=pd.DataFrame()

    for col, time in zip(
        dt_tmp["Activity_id"],dt_tmp.reset_index().Act_submit_Time.unique()
        ):
      df_bx=pd.DataFrame()
      df_bx=df_avt.query(
          f"Activity_id== '{col}' & Activity_type=='{chk}'"
          )[["Activity_id", "Act_mark"]]
      df_bx["Act_submit_Time"]= pd.to_datetime(time).strftime('%d-%m-%Y')
      df_stAct= pd.concat([df_stAct, df_bx ])

    sns.boxplot(x="Activity_id" ,  
                y= "Act_mark", data=df_stAct , ax=ax ,width= 0.3)
    
    dt_tmp.reset_index().append({
        "Act_submit_Time":ref_q.strftime('%d-%m-%Y'),
        "Act_mark":list(yq_point[1])[0]
    }, ignore_index=True).Act_mark.plot(ax=ax, color="k", marker='o', linewidth=2, markersize=9) 


    dt_tmp.reset_index().append({
        "Act_submit_Time":ref_q.strftime('%d-%m-%Y'),
        "Act_mark":list(yq_point[1])[0]
    }, ignore_index=True).tail(2).Act_mark.plot(ax=ax, color="r",  marker='o',linestyle='dashed' ,linewidth=5, markersize=14 ) 
    

    graphTitle= "Student's Assignement mark compared to other students" if chk  else " Student's quiz mark compared to other students" 
    plt.title(graphTitle , fontsize=20)
    plt.xlabel("Date" , fontsize=20)
    plt.ylabel("Mark" , fontsize=20)
    plt.xticks(rotation=70)
    ax.set_xticklabels(dt_tmp.index.strftime('%d-%m-%Y'))
    L=plt.legend()
    L.get_texts()[0].set_text('Actual marks')
    L.get_texts()[1].set_text('Predicted mark')

    plt.grid()
    plt.show()
  else:
    print("  No records Available")

In [58]:
@interact(st = stID3, cs = course3, dt = D3, chkbx = box3)
def plot_data2( st, cs, dt , chkbx):
  #print(((chkbx)+0))
  course3.options = std_d[st]
  box3.description = " Assignement"   if chkbx   else "Quizz"
  # print(predict_std.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}'"))
  plot_std_Data2(st, cs, dt , ((chkbx)+0))

interactive(children=(Dropdown(description='st', options=('080748524S', '090904691S', '060991839S', '071059051…

In [59]:
######  TRY PLOTLY (2)
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import datetime 

In [60]:
def plot_std_Data3(st, cs,dt, chk):

########## DATA HERE -----------------------
  dt_tmp= dt_st.query(
      f"student_ID=='{st}' & Course_code=='{cs}'& Activity_type=='{chk}'")[["Act_mark", "Activity_id" ]]
  dt_tmp=dt_tmp.query(f"Act_submit_Time >= '{dt}'")
  cmap = plt.cm.get_cmap('GnBu') # green to blue color way
  if dt_tmp.shape[0]>0:
    course4.options = std_d[st]
    
    dt_tmp.sort_index( inplace=True )
    dt_tmp.reset_index(inplace=True)
  #  Query only  records  done after  selected Date
    
  
    ref_t= dt_tmp.loc[dt_tmp.shape[0]-1, "Act_submit_Time"] + timedelta(days=7)
    dt_tmp['Act_submit_Time'] = dt_tmp.Act_submit_Time.apply(lambda x: x.strftime('%d-%m-%Y'))
  # dt_tmp["Act_submit_Time"]= dt_tmp.Act_submit_Time.astype(str).str[:-9]
  
######## Predicted value  ###########
  #   we suppose that  predict_std is already available
  
    yq_point=[dt_st.sort_index().query(
      f"student_ID=='{st}' & Course_code=='{cs}' & Activity_type=='{chk}' "
      ).Act_mark.tail(1) 
    , predict_std.reset_index().query(
       f"student_ID=='{st}'  & Course_code=='{cs}' & Activity_type=='{chk}' "
       ).last_mark]
     
 
########################################## 
    df_stAct=pd.DataFrame()
  

    for col, time in zip( dt_tmp["Activity_id"],dt_tmp.Act_submit_Time.unique()):
      df_bx=pd.DataFrame()
      df_bx=df_avt.query(
          f"Activity_id== '{col}' & Activity_type=='{chk}'"
          )[["Activity_id", "Act_mark"]]
      df_bx["Act_submit_Time"]= pd.to_datetime(time).strftime('%d-%m-%Y')
      df_stAct= pd.concat([df_stAct, df_bx ])
    
  #--------------New Predicted values------------------------------
    dt_plot=dt_tmp.tail(1)
    dt_plot= dt_plot.append({
        # "Act_submit_Time":pd.to_datetime(ref_t).strftime('%d-%m-%Y') , #.astype(str).str[:-9],
        "Activity_id": "_",
        "Act_mark":list(yq_point[1])[0]
    }, ignore_index=True)

    return dt_tmp, dt_plot , df_stAct 
  #_______________________PLOTLY___________________
 
  else:
    return " No records Available"


In [61]:
def   plot_dt(st, cs, dt, chk):
  dt_tmp, dt_plot, df_stAct= plot_std_Data3(st, cs, dt, chk)
  fig = go.Figure()
    
  fig.add_trace(
        go.Scatter(x=dt_tmp.Activity_id ,  y=dt_tmp.Act_mark,  
                  mode='lines+markers', name="درجة الطالب", marker_color="red"))
    

  dt_plot.Act_mark = round(dt_plot.Act_mark, 2)

  fig.add_trace(
        go.Scatter(x=dt_plot.Activity_id ,  y=dt_plot.Act_mark, line=dict(width=4, dash='dot'), marker=dict(size=12), 
                  mode='lines+markers', name="الدرجة المتوقعة", marker_color="blue"))
    

  fig.add_trace(
        go.Box(
            x=df_stAct.Activity_id ,  
            y= df_stAct.Act_mark,  name="توزيع درجات الطلاب", marker_color = "green", width =0.2, 
            # fillcolor=   'rgb' + str(cmap(norm(median))[0:3]) # normalize  
            ))


  fig.update_layout(
        autosize=False,
        width= 1200 , height= 600,
        
        hovermode="x" , 
        # title="أداء جميع الطلبة في مادة معينة",
      xaxis_title="Submit Time",
      yaxis_title="Mark",
      font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="RebeccaPurple"),
    
      xaxis = dict(
          # tickmode = 'array',
          tickvals = np.arange(dt_tmp.shape[0]),
          ticktext = dt_tmp.Act_submit_Time,
           tickangle = 45
      ),


      title={
          'text': "(مقارنة درجات الطالب (خطَي) مع درجات زملاءه (مخطط الصندوق ",
          'y':0.9,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top',
          }) 
  fig.show()


In [62]:
@interact(st = stID4, cs = course4, dt=D4, chkbx = box4)
def plot_data3( st, cs, dt, chkbx):
  box4.description = " عرض الأنشطة"   if chkbx   else "عرض الاختبارات"
  course4.options = std_d[st]

  # print(predict_std.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}'"))
  plot_dt(st, cs, dt,  ((chkbx)+0))

interactive(children=(Dropdown(description='st', options=('080748524S', '090904691S', '060991839S', '071059051…

In [63]:
# dt_st.loc["02-12-2020", "Act_mark"]
dt_st.query(f"Course_code=='g5_3' & student_ID=='080748524S' & Activity_type=='0' ")

,student_ID,Course_code,Activity_id,Activity_type,Unit_id,Chapter_id,Lesson_id,Act_diff,Act_cp_count,Act_time,Act_mark
Act_submit_Time,,,,,,,,,,,
2020-10-07 20:44:56,080748524S,g5_3,g5_3-Act_10,0,Un_5,Chp_0,Ls_1,3,2,12,4.88
2020-08-23 02:38:25,080748524S,g5_3,g5_3-Act_11,0,Un_7,Chp_4,Ls_9,2,2,16,9.73
2020-04-01 23:24:48,080748524S,g5_3,g5_3-Act_12,0,Un_3,Chp_7,Ls_3,3,1,4,0.61
2020-08-17 20:57:43,080748524S,g5_3,g5_3-Act_13,0,Un_6,Chp_0,Ls_0,3,1,13,7.25
2020-03-31 07:43:05,080748524S,g5_3,g5_3-Act_14,0,Un_2,Chp_1,Ls_3,4,2,11,2.31
2020-08-03 18:45:48,080748524S,g5_3,g5_3-Act_15,0,Un_0,Chp_2,Ls_1,4,2,8,4.20
2020-06-26 16:24:03,080748524S,g5_3,g5_3-Act_16,0,Un_7,Chp_5,Ls_6,1,2,3,1.64
2020-12-02 17:25:58,080748524S,g5_3,g5_3-Act_17,0,Un_5,Chp_2,Ls_0,2,1,14,3.79
2020-09-06 01:57:19,080748524S,g5_3,g5_3-Act_18,0,Un_4,Chp_1,Ls_1,1,1,6,5.37


In [101]:
def plt_mark_time(st, cs, dt, chk):
 
  dt_plot=dt_st.sort_index().reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}' & Activity_type=='{chk}' & Act_submit_Time>='{dt}' ")
  if (dt_plot.shape[0]>0):
    # dt_plot['Act_submit_Time'] = dt_plot.Act_submit_Time.apply(lambda x: x.strftime('%d-%m-%Y'))
    dt_plot.set_index("Act_submit_Time", inplace=True, drop=True)
    
 
    course5.options = std_d[st]
    
    #print(list(std_classPred.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}' & Activity_type=='{chk}' ").Predicted_class)[0])
    cc= list(std_classPred.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}' & Activity_type=='{chk}' ").Predicted_class)[0]
    

    fig = make_subplots(rows=2 , cols=1 , specs=[[{"secondary_y": True}],[{"secondary_y": True}]])

    
    ref_q=dt_st.reset_index().query(f"student_ID=='{st}' & Course_code=='{cs}' & Activity_type=='{chk}' ").Act_submit_Time.max()
    new_date = ref_q + timedelta(14)

    # ref_q=datetime.datetime.strftime(ref_q, '%d-%m-%Y')

    # new_date=datetime.datetime.strftime(new_date, '%d-%m-%Y')



    x_point=[ref_q, new_date]

    y_point2= list(predict_std.reset_index().query(f"student_ID=='{st}'  & Course_code=='{cs}' & Activity_type=='{chk}' ")["last_mark"])[0]
    y_point2 = str(round(y_point2, 2))

    y_point=[dt_st.loc[ref_q, "Act_mark"], y_point2]
 
     
    #----------- Graph formatting  ---------------------------------

    fig.add_trace(
    go.Scatter(x=dt_plot.index, y=dt_plot.Act_mark, name="الدرجة"),
    secondary_y=False, row=1, col=1)

    fig.add_trace(
    go.Scatter(x=dt_plot.index, y=dt_plot.Act_time, name="مدة التسليم"),
    secondary_y=True, row=1, col=1 )

    fig.add_trace(
    go.Scatter(x=x_point, y=y_point, name="الدرجة المتوقعة",line=dict(width=4, dash='dot'),
               marker=dict(size=12)), row=1, col=1)



    fig.update_layout(hovermode="x")

    fig.update_yaxes(row=1, col=1   ,   showgrid=True,title_text="الدرجة",title_font_size=20, secondary_y=False)
    fig.update_yaxes(row=1, col=1, showgrid=False,title_text="مدة تسليم التقييم / بالدقائق",title_font_size=20, 
                     secondary_y=True)
    
    fig.add_annotation(dict(font=dict(color='black',size=18),
                                        x=-0.07,
                                        y=-0.45,
                                        showarrow=False,
                                        text="قد ترغب في متابعة التوصيات التالية",
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper",
                                        bgcolor='white',
                                        bordercolor="purple",
                                        borderwidth=3,borderpad=7))
    
    fig.add_annotation(dict(font=dict(color='black',size=15),
                                        x=0.88,
                                        y=1.3,
                                        showarrow=False,
                                        text="الدرجة المتوقعة",
                                        textangle=0,
                                        xanchor='center',
                                        xref="paper",
                                        yref="paper",
                                        bgcolor='white',
                                        bordercolor="purple",
                                        borderwidth=3, borderpad=4))
    
    fig.add_annotation(dict(font=dict(color='black',size=15),
                                        x=0.788,
                                        y=1.3,
                                        showarrow=False,
                                        text=y_point2,
                                        textangle=0,
                                        xanchor='center',
                                        xref="paper",
                                        yref="paper",
                                        bgcolor='white',
                                        bordercolor="purple",
                                        borderwidth=3,
                                        borderpad=4
                                        ))
    
    fig.add_annotation(dict(font=dict(color='black',size=15),
                                        x=0.88,
                                        y=1.15,
                                        showarrow=False,
                                        text="التصنيف المتوقع",
                                        textangle=0,
                                        xanchor='center',
                                        xref="paper",
                                        yref="paper",
                                        bgcolor='white',
                                        bordercolor="purple",
                                        borderwidth=3,
                                        borderpad=4))
    
    fig.add_annotation(dict(font=dict(color='black',size=15),
                                        x=0.8,
                                        y=1.15,
                                        showarrow=False,
                                        text=cc,
                                        textangle=0,
                                        xanchor='center',
                                        xref="paper",
                                        yref="paper",
                                        bgcolor='white',
                                        bordercolor="purple",
                                        borderwidth=3,
                                        borderpad=4
                                                                               
                                        ))
    
    
  #________________________  Graph2_________________
    if not isinstance(plot_std_Data3(st, cs, dt, chk), str):
      dt_tmp, dt_plot, df_stAct= plot_std_Data3(st, cs, dt, chk)
      # fig = go.Figure()
    
      fig.add_trace(
        go.Scatter(x=dt_tmp.Activity_id ,  y=dt_tmp.Act_mark,  
                  mode='lines+markers', name="درجة الطالب", marker_color="red"), row=2, col=1)
    

      dt_plot.Act_mark = round(dt_plot.Act_mark, 2)

      fig.add_trace(
        go.Scatter(x=dt_plot.Activity_id ,  y=dt_plot.Act_mark, line=dict(width=4, dash='dot'), marker=dict(size=12), 
                  mode='lines+markers', name="الدرجة المتوقعة", marker_color="blue"), row=2, col=1)
    

      fig.add_trace(
        go.Box(
            x=df_stAct.Activity_id ,  
            y= df_stAct.Act_mark,  name="توزيع درجات الطلاب", marker_color = "green", width =0.2, 
            # fillcolor=   'rgb' + str(cmap(norm(median))[0:3]) # normalize  
            ), row=2, col=1)


      fig.update_layout(
        autosize=False,
        width= 1200 , height= 600,
        
        hovermode="x" , 
        # title="أداء جميع الطلبة في مادة معينة",
      xaxis_title="Submit Time",
      yaxis_title="Mark",
      font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="RebeccaPurple"),
    
      xaxis = dict(
          # tickmode = 'array',
          tickvals = np.arange(dt_tmp.shape[0]),
          ticktext = dt_tmp.Act_submit_Time,
           tickangle = 45
      ),


      title={
          'text': "(مقارنة درجات الطالب (خطَي) مع درجات زملاءه (مخطط الصندوق ",
          'y':0.9,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top',
          }) 

  #______________________________end Graph2

    
      fig.update_layout(
        autosize=False,
        width= 1200 , height= 600,
      title={
        'text': "مقارنة درجات الطالب مع مدة تسليم التقييم",
        'y':0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'},
        font=dict(color='black',size=18),
      xaxis = dict(
       
           tickangle = 45
      ))
    

    
      fig.show()
    else:
      print(" No records Available")

#### printing URLs for recommendatios 

    for item in recommend(st):
      display(HTML(f"<a href={url_lobj(item)} target='_blank' style='font-size:1.3em'>{item}</a>"))
    
  else:
    print(" No records Available")
    

In [102]:
ref_q=dt_st.reset_index().query(f"student_ID=='080748524S' & Course_code=='g5_3' & Activity_type=='0' ").Act_submit_Time.max()
dt_st.loc[ref_q, "Act_mark"]

3.79

In [103]:
not isinstance("samir", str)


False

In [104]:
@interact(st = stID5, cs = course5, dt = D5, chkbx = box5)
def plotly_data( st, cs, dt , chkbx):
  #print(((chkbx)+0))

  course5.options = std_d[st]
  box5.description = " عرض الأنشطة"   if chkbx   else "عرض الاختبارات"

  # print(predict_std.reset_index().query(f"Course_code=='{cs}' & student_ID=='{st}'"))
  plt_mark_time(st, cs, dt , ((chkbx)+0))
  # plot_std_Data3(st, cs,dt,  ((chkbx)+0) )
  plot_strec( st,cs,  dt)

interactive(children=(Dropdown(description='st', index=2, options=('080748524S', '090904691S', '060991839S', '…

## Plot submit time Vs other student submitions ( with box plot )

## Exporting to HTML

In [70]:
###### TRY EXPORTING TO HTML
#### Look for the example below

#https://medium.com/analytics-vidhya/how-to-export-a-plotly-chart-as-html-3b5df568df4a

In [ ]:
import plotly.express as px
sepal_plot = px.scatter(data_frame=df, x=df.sepal_length, y=df.sepal_width, color='species', title="Iris_sepal")

In [ ]:
sepal_plot

In [ ]:
petal_plot = px.scatter(data_frame=df, x=df.petal_length, y=df.petal_width, color='species', title="Iris_petal")

In [ ]:
petal_plot

In [ ]:
plotly.offline.plot(petal_plot, filename='Data/pettal.html')

In [ ]:
button = widgets.Button(description='My Button')


In [ ]:
button

In [ ]:
button = widgets.Button(description='My Button' )
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          #clear_output()
       
          print('Something happens!')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

In [ ]:
#testing adding button and textbox
from ipywidgets import widgets as wd
from ipywidgets import Layout, Box
from IPython.display import display

btnLayout = Layout(flex='1 1 auto', width='auto')

btns= [
    wd.Button(description="Add Route"),
    wd.Button(description="234"),
    wd.Button(description="411")
]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch')

box = Box(children=btns, layout=box_layout)

display(box)
box.children[1].layout.visibility = 'hidden'
box.children[2].layout.visibility = 'hidden'

txtArea = wd.Textarea()
display(txtArea)

def show(b):
    box.children[1].layout.visibility = 'visible'
    box.children[2].layout.visibility = 'visible'

#function intended to append button value/description to textArea
def add_text(b):
    txtArea.value = b.description + txtArea.value

box.children[0].on_click(show)
box.children[1].on_click(add_text)
box.children[2].on_click(add_text)

In [ ]:
# imports
import plotly
import plotly.express as px

# data
df = px.data.gapminder().query("continent=='Oceania'")

# plotly express bar chart
fig = px.line(df, x="year", y="lifeExp", color='country')

# html file
plotly.offline.plot(fig, filename='Data/lifeExp.html')

# Data sets creation

## 1 ) Create Student's General Info. Dataset

In [ ]:
grades={
    "06":"9","07":"8",
    "08":"7", "09":"6", "10":"5", "11":"4",
    "12":"3", "13":"2", "14":"1"
}

In [ ]:
###  الطلبة العينة من 10 مدارس عشوائيًا (500 طالب) ##
##### من الصف السادس الى الثالث الاعدادي 


list_std=[]
for i in range(500):
  birth_year=random.choice(["06", "07", "08", "09","10"]) 
  student= {
  "student_id": ''.join(random.choice(string.digits) for _ in range(5)) ,
  "birth_year": birth_year,
  "birth_month": random.choice(["01", "02", "03", "04", "05", 
                                "06", "07", "08", "09","10","11","12"]),
  "school": random.choice([ "Sc_"+str(j) for j in range(10)]),
  "grade": grades[birth_year],  
  "gender": random.choice(["M", "F"]),
  "governorate": random.choice(["North", "South", "Caiptal", "Muharraq"])
         } 
  list_std.append(student)     
df_st=pd.DataFrame(list_std)


In [ ]:
#####  Merge student_ID with birth year and month month to get a complete CPR

df_st.insert(0, 'student_ID', df_st['birth_year'].astype(str) + df_st['birth_month'].astype(str)+df_st.pop('student_id').astype(str))

In [ ]:
df_st.head()

In [ ]:
for col in df_st.keys():
  df_st[col]= df_st[col].astype(str)

In [ ]:
df_st.info()

In [ ]:
df_st.to_csv("./Data/General_students_info.csv")

## 2 ) Create Students' portal activity Dataset / df_avt

In [ ]:
from random_timestamp import random_timestamp
lst=[]
for i, k  in zip(df_st.student_ID, df_st.grade):
  for j in range (14, int((20*np.random.rand()+8))):
    course_id=random.choice(["g"+str(k)+"_"+str(l) for l in range(1,4)])
    ActType=random.choice(["1", "0" ])
    Act_time= [15, 2]  if ActType=='0'  else [4320, 60]
    activities= {
        "student_ID": i ,
        "Course_code":course_id,  
        "Activity_id": f"{course_id}-Act_{int(20*np.random.rand()+1)}",
        "Activity_type":ActType,
        # "Act_count": course_ref[course_id][ActType] , #np.random.randint(5,30),

      "Unit_id":  random.choice([ "Un_"+str(k) for k in range(10)]),
      "Chapter_id": random.choice([ "Chp_"+str(k) for k in range(10)]), 
      "Lesson_id": random.choice([ "Ls_"+str(k) for k in range(10)]),
      # "Competence-id":  random.choice([ "Cp_"+str(k) for k in range(10)]), 
      "Act_diff":  int((4*np.random.rand(1)+1)),
      "Act_cp_count": int((np.random.rand(1)*2+1)), 

      "Act_time": int((Act_time[0]*np.random.rand()+Act_time[1])) , ####### Quizzes from 2 mins to 15 mins // Assign. from 1 hr to 3 days
      "Act_submit_Time": random_timestamp( year=2020),
      #  add  afeature for  late submission
      "Act_mark": int((1000*np.random.rand()))/100,
    }
    lst.append(activities)
df_avt=pd.DataFrame(lst)

In [ ]:
df_avt.drop_duplicates(subset=["student_ID"	, 	"Activity_id"	, "Activity_type"] , keep="last", inplace=True)

In [ ]:
df_avt.sample(10)

In [ ]:
df_avt.info()

In [ ]:
df_avt.to_csv("./Data/Portal-Achiev-Info.csv")

## 3 ) Generate 5 selected Students Data to graph  / dt_st


In [ ]:
def dataJson(actType, i , c, act):
  Act_time= [15, 2]  if actType=='0'  else [4320, 60]
  return {
      
        "student_ID": i ,
        "Course_code":c,  
        "Activity_id": act,
        "Activity_type":actType,
          

        "Unit_id":  random.choice([ "Un_"+str(k) for k in range(10)]),
        "Chapter_id": random.choice([ "Chp_"+str(k) for k in range(10)]), 
        "Lesson_id": random.choice([ "Ls_"+str(k) for k in range(10)]),
        "Act_diff":  int((4*np.random.rand(1)+1)),
        "Act_cp_count": int((np.random.rand(1)*2+1)), 

        "Act_time": int((Act_time[0]*np.random.rand()+Act_time[1])) , ####### from 30 SEC to 15 MIN
        "Act_submit_Time":random_timestamp( year=2020),  
        #  add  a feature for  late submission
        "Act_mark": int((1000*np.random.rand()))/100,  #  this should be  float32  : 10*np.random.rand()
      }

In [ ]:
std_course=[['g5_3', 'g5_2', 'g5_1'], ['g9_3', 'g9_1', 'g9_2'],['g6_1', 'g6_3',
       'g6_2'], ['g8_1', 'g8_3', 'g8_2'], ['g7_1',  'g7_3', 'g7_2'] ]

In [ ]:
from random_timestamp import random_timestamp
lst=[]
std_array= df_avt.student_ID.unique()[:5] +"S"
for i , course in zip(std_array, std_course):# c in df_avt.Course_code.unique():
  for c in course:
    for j in [f"{c}-Act_{act}"  for act in range(10,19)]: #range (1, int((2*np.random.rand()+8))):
      lst.append(dataJson("0", i , c, j ))
    for j in [f"{c}-Act_{act}"  for act in range(1,5)]:
      lst.append(dataJson("1", i , c, j ))
     
dt_st=pd.DataFrame(lst)

In [ ]:
for s, c  in  zip(dt_st.student_ID.unique(), std_course[0]):
  count=dt_st.query(f"student_ID=='{s}' & Course_code=='{c}' & Activity_type=='1'").shape[0]
  print(f"Student:  {s}  has: {count}  records")

In [ ]:
dt_st.sample(5)

In [ ]:
dt_st.to_csv("./Data/5_selected_students.csv")  

## Create Recommendation data set

In [ ]:
def url_lobj(lob):
  
  for i in modules:
    for j in i["lessons"]:
      for k in j["lobs"]:
        if k["lobName"]==lob:
          return  k["URL"]


In [ ]:
url_lobj('Representing Data')

In [ ]:
#  Json  recommendation Initialization
modules=[
         {"unit":"Data and statistics", 
          "lessons":[
                    {"LessName": "Dot plots & frequency tables",
                    "lobs":[
                            {"lobID": 1, "lobName":"Representing Data", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/dot-plot/v/ways-to-represent-data?modal=1" },
                            {"lobID": 2,"lobName":"Frequency Tables & dot plot", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/dot-plot/v/frequency-tables-and-dot-plots?modal=1"}
                          ]
                          
                    },
                    {"LessName": "Statistical questions",
                    "lobs":[
                            {"lobID": 3, "lobName":"Statistical questions", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-statistical-questions/v/understanding-statistical-questions?modal=1"}
                          ]
                    },
                    {"LessName": "Histograms",
                    "lobs":[
                            {"lobID": 4, "lobName":"Creating a histogram", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/histograms/v/histograms-intro?modal=1"},
                            {"lobID": 5, "lobName":"Interpreting a histogram", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/histograms/v/interpreting-histograms?modal=1"}
                          ]
                    },
                    {"LessName": "Mean and median",
                    "lobs":[
                            
                                {"lobID": 6, "lobName":"Statistics Intro: Mean, Median & Mode", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/mean-and-median/v/statistics-intro-mean-median-and-mode?modal=1"}, 
                                {"lobID": 7, "lobName":"Mean, Median & Mode example", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/mean-and-median/v/mean-median-and-mode?modal=1"},
                                {"lobID": 8 , "lobName":"Calculating the mean", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/mean-and-median/a/calculating-the-mean?modal=1"}
                                
                                
                          ]
                    },
                    {"LessName": "Mean and median challenge problems",
                    "lobs":[
                             {"lobID": 9, "lobName": "Missing value given the mean", "URL": "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/using-mean-to-find-missing-value?modal=1"},
                              {"lobID":10 , "lobName":"Mean as the balancing point" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/a/mean-as-the-balancing-point?modal=1"},
                              {"lobID": 11, "lobName":"Impact on median & mean: Removing an outlier" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/impact-on-median-and-mean-when-removing-lowest-value-example?modal=1"},
                              {"lobID": 12, "lobName":"Impact on median & mean: increasing an outlier" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/impact-on-median-and-mean-when-increasing-highest-value?modal=1"},
                              {"lobID": 13, "lobName":"Median & range puzzlers" , "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-mean-median-challenge/v/median-and-range-puzzle?modal=1"}
                          ]
                    },
                    {"LessName": "Interquartile range (IQR)",
                    "lobs":[
                            {"lobID": 14, "lobName":"Interquartile range (IQR)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th/v/calculating-interquartile-range-iqr?modal=1"}
                          ]
                    },
                    {"LessName": "Box Plots",
                    "lobs":[
                            {"lobID":15  , "lobName":"Reading box plots", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/reading-box-and-whisker-plots?modal=1"},
                             {"lobID":16  , "lobName":"Constructing a box plot", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/constructing-a-box-and-whisker-plot?modal=1"},
                             {"lobID":17  , "lobName":"Worked example: Creating box plot (odd number of data points)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/box-and-whisker-plot-exercise-example?modal=1"},
                             {"lobID":18  , "lobName":"Worked example: Creating box plot (even number of data points)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/constructing-a-box-and-whisker-plot?modal=1"},
                             {"lobID":19  , "lobName":"Worked example: Interpreting a box plot", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6th-box-whisker-plots/v/interpreting-box-plots?modal=1"}
                          ]
                    },

                    {"LessName": "Mean absolute deviation (MAD)",
                    "lobs":[
                            { "lobID":20 , "lobName":"Mean absolute deviation (MAD)", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-mad/v/mean-absolute-deviation?modal=1"},
                              {"lobID":21 , "lobName":"Mean absolute deviation example", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-mad/v/mean-absolute-deviation-example?modal=1"}
                          ]
                    },
                    {"LessName": "Comparing Data Display",
                    "lobs":[
                            {"lobID":22 , "lobName":"Comparing dot plots, histograms and box plots", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-7th-compare-data-displays/v/comparing-dot-plots-histograms-and-box-plots?modal=1"}
                          ]
                    },
                    {"LessName": "Shape of Data Distribution",
                    "lobs":[
                            {"lobID":23 , "lobName":"Shape of distributions", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-shape-of-data/v/shapes-of-distributions?modal=1"},
                              {"lobID":24 , "lobName":"Clusters, gaps, peaks & outliers", "URL" : "https://www.khanacademy.org/math/cc-sixth-grade-math/cc-6th-data-statistics/cc-6-shape-of-data/v/examples-analyzing-clusters-gaps-peaks-and-outliers-for-distributions?modal=1"}
                          ]
                    },
                 ],        
          }
         ]

In [ ]:
lst=[]
md=modules[0]["lessons"]

for st in  dt_st.student_ID.unique():
  for i   in range(int(30*np.random.rand()+8)):
    RndLesson= int(len(md)*np.random.rand()-1)
    RndLob= int(len(md[RndLesson]["lobs"])*np.random.rand())
    activities= {
        "student_ID": st ,
        # "Course_code":course_id,  
        # "Unit_des":  random.choice([ "Un_"+str(k) for k in range(10)]),
        "Lesson_des": md[RndLesson]["LessName"],

        "lobj_id": md[RndLesson]["lobs"][RndLob]["lobID"], 
        "lobj_des": md[RndLesson]["lobs"][RndLob]["lobName"],  
        # "Subcps": [ subcp[s]  for   i in range (1, int((len(subcps)*np.random.rand()+2)))] 

    }
    lst.append(activities)
df_recommend=pd.DataFrame(lst)

In [ ]:
df_recommend["LearningObj"]=df_recommend.Lesson_des+" "+ df_recommend.lobj_des

In [ ]:
df_recommend.to_csv("./Data/recommend.csv")

## Data Preparation for prediction

In [ ]:
df_avt.tail()

In [ ]:
dt_st.tail()

In [ ]:
#  Add  5 selected student used  in the Graph Above

#df_avt= df_avt.drop("Act_submit_Time", axis=1).reset_index().append( dt_st.reset_index())


df_avt= df_avt.append(dt_st.reset_index())

In [ ]:
df_avt.drop([  "Unit_id", "Chapter_id", "Lesson_id"], inplace=True, axis=1)

In [ ]:
df_avt.tail()

In [ ]:
#  4  dashboard
#plt.rcParams.update({'font.size': 12})
#df_avt[["Act_mark"]].groupby(pd.Grouper(freq="W")).agg("mean").plot()

In [ ]:
df_act_Stat=pd.pivot_table(df_avt, index=["Course_code","Activity_type" ], 
               values=["Act_mark", "Act_diff", "Act_cp_count", "Act_time" , "Activity_id"]   , aggfunc={
                   "Act_mark":("mean","max", "count", "min"), 
                   "Act_diff": "mean",
                   "Act_cp_count": "mean",
                   "Act_time": ("mean", "max", "min"),
                   "Activity_id": lambda x: x.nunique()
               }).rename(columns={
                   "Activity_id" :  "Activities_Count"  #  number of activities
#                 "Act_mark":"Std_act_m", 
#                    "Act_diff": "Std_act_d",
#                    "Act_cp_count": "Std_act_cp",
#                    "Act_time": "Std_act_t"  

               })


In [ ]:
df_avt.head(10)

In [ ]:
df_avt.query(f"student_ID=='090904691'").groupby(["Course_code", "Activity_type"]).student_ID.agg("count")

In [ ]:
df_avt.groupby(["Course_code", "Activity_type"]).student_ID.agg("count")

In [ ]:
df_avt[df_avt.Course_code=='g8_1'].groupby("Activity_type").student_ID.agg("count")

In [ ]:
df_act_Stat.sample(5)

In [ ]:
df_act_Stat.columns=[f"{i}_{j}" for  i , j in df_act_Stat.columns]

In [ ]:
df_avt[(df_avt.Course_code=="g6_1")&(df_avt.Activity_type=="0")].describe()

## Students' Achievement Statistics (( df_std ))  (( df_mg ))

In [ ]:
df_std_Stat=pd.pivot_table(df_avt, index=["student_ID","Course_code","Activity_type" ], 
               values=["Act_mark", "Act_diff", "Act_cp_count", "Act_time" ]   , aggfunc={
                   "Act_mark":("mean","max", "count", np.std, "last"), 
                   "Act_diff": "mean",
                   "Act_cp_count": "mean",
                   "Act_time": ("mean", "max", np.std)
               },  fill_value=0 ).rename(columns={
                "Act_mark":"Std_act_m", 
                   "Act_diff": "Std_act_d",
                   "Act_cp_count": "Std_act_cp",
                   "Act_time": "Std_act_t"   

               })
df_std_Stat.head()

In [ ]:
df_avt.sample()

In [ ]:
df_avt[(df_avt.student_ID=="080607421")& (df_avt.Course_code=="g7_3")]

In [ ]:
df_std_Stat.columns=[f"{i}_{j}" for  i , j in df_std_Stat.columns]

In [ ]:
df_std_Stat.sample(5)

- Ranking

$$\Large score=\log(\frac{\text{Qz_Max_time}}{\text{Qz_Avg_tim}}+\text{Qz_avg_diff*Qz_Avg_cp})*\frac{\text{Qz_Avg_mark}^2}{\text{Qz_Max_mark}}$$

In [ ]:
df_std_Stat["Act_score"]=np.log1p( 
    df_std_Stat.Std_act_t_max/df_std_Stat.Std_act_t_mean +df_std_Stat.Std_act_d_mean* df_std_Stat.Std_act_cp_mean
)*df_std_Stat["Std_act_m_max"]**2/df_std_Stat.Std_act_m_mean

In [ ]:
# reset index   for  Students    data
df_std_Stat.reset_index(inplace=True)
# reset index for   activities data
df_act_Stat.reset_index(inplace=True)

In [ ]:
# Merge Data
df_mg=df_std_Stat.merge(df_act_Stat,  on=["Course_code", "Activity_type"])

In [ ]:
df_mg.query(f"student_ID=='080554476'")

In [ ]:
df_mg["last_mark"]=df_mg.Std_act_m_last  # put the target at the end of table
df_mg.drop("Std_act_m_last", axis=1, inplace=True)  #  drop the last mark feature.

In [ ]:
df_mg.sample(3)

In [ ]:
df_mg.info()

In [ ]:
df_mg.to_csv("./Data/student_statistics.csv")

# Linear Regression & Classification

## Pipeline    SGB Regressor  Mark Prediction 
![](https://i.ibb.co/xzWVv5H/pipe-Lasso.png)

In [ ]:
df_mg.set_index("student_ID", inplace=True)

In [ ]:
# Split Data

X_train=df_mg.drop(dt_st.student_ID.unique(), axis = 0).iloc[:,:-1]
y=df_mg.drop(dt_st.student_ID.unique(), axis = 0).iloc[:, -1]

In [ ]:
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.compose import make_column_transformer , make_column_selector  as selector

from sklearn.pipeline import make_pipeline
  
num_pipeline= make_pipeline(
     StandardScaler( )    # Rescaling
)

cat_pipeline= make_pipeline(
      
     OneHotEncoder(drop="if_binary" , sparse=False ), 
 
)

In [ ]:
#  Pipeline structure 
preprocess= make_column_transformer(
     ( num_pipeline,  selector(dtype_include="number") ),  #Numeric feature rescaling
     (cat_pipeline, selector(dtype_exclude="number"))  # Categorical features Encoding
     )

In [ ]:
preprocess.fit_transform(X_train).shape

In [ ]:
preprocess.fit_transform(X_train)[0]

In [ ]:
# pipelline Model

from xgboost import XGBRegressor
pipe_reg= make_pipeline(
    preprocess, 
    XGBRegressor(objective="reg:squarederror")
    )

In [ ]:
pipe_reg.get_params()

In [ ]:
# hyper parameters

param_grid={
    "xgbregressor__learning_rate": [0.01, 0.1],
    "xgbregressor__max_depth" : [3,5,7, 10] ,
    "xgbregressor__min_child_weight" :[1,3,5],
    "xgbregressor__subsample": [0.5, 0.7],
    "xgbregressor__colsample_bytree": [0.5, 0.7],
    "xgbregressor__n_estimators": [200, 400, 1000]
    
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold

kf= KFold( n_splits=5, shuffle=True )
gs= RandomizedSearchCV( pipe_reg, param_distributions=param_grid,  scoring="neg_mean_squared_error", cv=kf,verbose=3)

In [ ]:
gs.fit(X_train, y)

In [ ]:
gs.best_estimator_

In [ ]:
gs.best_params_

In [ ]:
df_mg.loc[dt_st.student_ID.unique(), :].iloc[:, :-1].sample()

In [ ]:
y_pred=gs.predict(X_train)

In [ ]:
from sklearn import  metrics
print(f"  MAE : { metrics.mean_absolute_error(y, y_pred):.3f}")
print(f"  MSE : { metrics.mean_squared_error(y, y_pred):.3f}")
print(f"  RMSE : { np.sqrt(metrics.mean_squared_error(y, y_pred)):.3f}")
print(f"  R^2 : { metrics.r2_score(y, y_pred):.3f}")
print(f"  Explained Variance Score : { metrics.mean_absolute_error(y, y_pred):.3f}")

In [ ]:
# Residual Plot

import seaborn as sns
_= sns.residplot(x=y, y=y_pred, color="green")

In [ ]:
predict_std = df_mg.loc[dt_st.student_ID.unique(), :].iloc[:, :-1]

In [ ]:
gs.predict(predict_std)

In [ ]:
predict_std["last_mark"]=gs.predict(predict_std)

In [ ]:
predict_std.head()

In [ ]:
predict_std.to_csv("./Data/predicted_data.csv")

## Pipeline    SGB Classsifier  Classification  Prediction   ((( ERROR ))
![](https://i.ibb.co/xzWVv5H/pipe-Lasso.png)

In [ ]:
df_mg.sample()

,student_ID,Course_code,Activity_type,Std_act_cp_mean,Std_act_d_mean,Std_act_m_count,Std_act_m_max,Std_act_m_mean,Std_act_m_std,Std_act_t_max,Std_act_t_mean,Std_act_t_std,Act_score,Act_cp_count_mean,Act_diff_mean,Act_mark_count,Act_mark_max,Act_mark_mean,Act_mark_min,Act_time_max,Act_time_mean,Act_time_min,Activities_Count_<lambda>,last_mark
633,070415152,g8_2,1,1.25,2.75,4,6.19,3.955,2.717603,3907,2253.25,1614.54955,14.073199,1.578125,2.734375,64,9.81,5.080781,0.38,4116.0,2171.1875,71.0,20,6.16


In [ ]:
df_stClass= df_mg.copy()
df_stClass.set_index("student_ID", inplace=True, drop=True)


In [ ]:
df_stClass.drop(std_d, axis=0, inplace=True)

- Ranking

$$\Large score=\log(\frac{\text{Qz_Max_time}}{\text{Qz_Avg_tim}}+\text{Qz_avg_diff*Qz_Avg_cp})*\frac{\text{Qz_Avg_mark}^2}{\text{Qz_Max_mark}}$$

In [ ]:
df_stClass["Activity_type"]=df_stClass.Activity_type.astype("category")

In [ ]:
#  feature  Engineering

bins = [0, 
        df_stClass.Act_score.quantile(0.25),
        df_stClass.Act_score.quantile(0.5),
        df_stClass.Act_score.quantile(0.75), 
        df_stClass.Act_score.quantile(1)]
names = ['D', 'C', 'B', 'A']

df_stClass['Score_Class'] = pd.cut(df_stClass.Act_score,
                           bins, labels=names, include_lowest=True)

In [ ]:
df_stClass.drop(["Act_score"], axis=1, inplace=True)

In [ ]:
df_stClass.Score_Class.unique()
#  It's better to add data related to  timestamp feature:  dat, month, dayofteweek, monthOfTheyear .....

['D', 'A', 'B', 'C']
Categories (4, object): ['D' < 'C' < 'B' < 'A']

In [ ]:
# Split Data

X=df_stClass.iloc[:,:-1]
y=df_stClass["Score_Class"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X, y  , test_size=.3,   random_state=42)

In [ ]:
X_test.head()

,Course_code,Activity_type,Std_act_cp_mean,Std_act_d_mean,Std_act_m_count,Std_act_m_max,Std_act_m_mean,Std_act_m_std,Std_act_t_max,Std_act_t_mean,Std_act_t_std,Act_cp_count_mean,Act_diff_mean,Act_mark_count,Act_mark_max,Act_mark_mean,Act_mark_min,Act_time_max,Act_time_mean,Act_time_min,Activities_Count_<lambda>,last_mark
student_ID,,,,,,,,,,,,,,,,,,,,,,
091095641,g6_3,0,1.000000,1.500000,2,5.10,2.575,3.570889,7,4.500000,3.535534,1.506024,2.578313,83,9.80,4.910602,0.00,16.0,8.530120,2.0,20,5.10
070468879,g8_2,0,1.666667,2.666667,3,8.84,6.020,2.582615,5,3.666667,1.527525,1.500000,2.573529,68,9.88,4.988971,0.02,16.0,8.411765,2.0,19,3.77
090361770,g6_1,1,2.000000,2.000000,1,5.95,5.950,0.000000,3386,3386.000000,0.000000,1.493506,2.493506,77,9.59,4.792857,0.08,4257.0,1978.792208,164.0,20,5.95
070645247,g8_3,0,2.000000,4.000000,1,4.79,4.790,0.000000,12,12.000000,0.000000,1.492308,2.600000,65,9.92,4.850308,0.18,16.0,9.723077,2.0,20,4.79
081011467,g7_1,1,2.000000,4.000000,1,8.28,8.280,0.000000,3599,3599.000000,0.000000,1.534091,2.386364,88,9.91,5.433182,0.04,4341.0,2159.340909,140.0,20,8.28


In [ ]:
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.compose import make_column_transformer , make_column_selector  as selector

from sklearn.pipeline import make_pipeline
  
num_pipeline= make_pipeline(
     StandardScaler( )    # Rescaling
)

cat_pipeline= make_pipeline(
      
     OneHotEncoder(drop="if_binary" , sparse=False ), 
 
)

In [ ]:
#  Pipeline structure 

preprocess= make_column_transformer(
     ( num_pipeline,  selector(dtype_include="number") ),  #Numeric feature rescaling
     (cat_pipeline, selector(dtype_exclude="number"))  # Categorical features Encoding
     )

In [ ]:
preprocess.fit_transform(X_train).shape

(910, 36)

In [ ]:
preprocess.fit_transform(X_train)[0]

array([-1.21442672, -0.55172367, -0.79989355, -0.95457584, -0.6513625 ,
       -0.75980405, -0.84435533, -0.83376001, -0.47616855, -1.06327502,
       -0.89395344,  0.95934381,  0.88321657, -0.16262916, -0.49749329,
       -1.01531287, -1.0126153 , -0.89800358, -1.41491831, -0.55880438,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])

In [ ]:
# pipelline Model

from xgboost import XGBClassifier
pipe_reg= make_pipeline(
    preprocess, 
    XGBClassifier()
    )

In [ ]:
pipe_reg.get_params()

{'columntransformer': ColumnTransformer(transformers=[('pipeline-1',
                                  Pipeline(steps=[('standardscaler',
                                                   StandardScaler())]),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f08562def10>),
                                 ('pipeline-2',
                                  Pipeline(steps=[('onehotencoder',
                                                   OneHotEncoder(drop='if_binary',
                                                                 sparse=False))]),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f08562dee10>)]),
 'columntransformer__n_jobs': None,
 'columntransformer__pipeline-1': Pipeline(steps=[('standardscaler', StandardScaler())]),
 'columntransformer__pipeline-1__memory': None,
 'columntransformer__pipeline-1__standardscaler': StandardScaler(),
 'columntransformer__pip

In [ ]:
# hyper parameters

param_grid = {
    'xgbclassifier__n_estimators': [50, 100, 150, 200],
    'xgbclassifier__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'xgbclassifier__max_depth': range(3, 5),
    'xgbclassifier__colsample_bytree': [i/10.0 for i in range(1, 3)],
    'xgbclassifier__gamma': [i/10.0 for i in range(3)],
    
}

In [ ]:
# AUC and accuracy as score

from sklearn.metrics import accuracy_score, make_scorer, f1_score
scorer = make_scorer(f1_score, pos_label="yes", average="micro")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold

kf= KFold( n_splits=5, shuffle=True )
gs= RandomizedSearchCV( pipe_reg, param_distributions=param_grid,  scoring=scorer,  cv=kf,verbose=3)

In [ ]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END xgbclassifier__colsample_bytree=0.1, xgbclassifier__gamma=0.1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=4, xgbclassifier__n_estimators=100;, score=0.676 total time=   0.2s
[CV 2/5] END xgbclassifier__colsample_bytree=0.1, xgbclassifier__gamma=0.1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=4, xgbclassifier__n_estimators=100;, score=0.687 total time=   0.2s
[CV 3/5] END xgbclassifier__colsample_bytree=0.1, xgbclassifier__gamma=0.1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=4, xgbclassifier__n_estimators=100;, score=0.681 total time=   0.2s
[CV 4/5] END xgbclassifier__colsample_bytree=0.1, xgbclassifier__gamma=0.1, xgbclassifier__learning_rate=0.01, xgbclassifier__max_depth=4, xgbclassifier__n_estimators=100;, score=0.714 total time=   0.2s
[CV 5/5] END xgbclassifier__colsample_bytree=0.1, xgbclassifier__gamma=0.1, xgbclassifier__learning_rate=0.01, xgbclassifie

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('standardscaler',
                                                                                                StandardScaler())]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x7f08562def10>),
                                                                              ('pipeline-2',
                                                                               Pipeline(steps=[('onehotencoder',
                                                                                                One...
                                                             

In [ ]:
gs.best_estimator_

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f085678e190>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f085678e990>)])),
                ('xgbclassifier',
                 XGBClassifier(colsample_bytree=0.2, gamma=0.2,
      

In [ ]:
gs.best_params_

{'xgbclassifier__colsample_bytree': 0.2,
 'xgbclassifier__gamma': 0.2,
 'xgbclassifier__learning_rate': 0.2,
 'xgbclassifier__max_depth': 3,
 'xgbclassifier__n_estimators': 200}

In [ ]:
std_classPred=df_mg.set_index("student_ID").loc[std_d, :].drop("Act_score", axis=1)

In [ ]:
# gs.predict(X_test.loc["061108800"])
std_classPred["Predicted_class"]= gs.predict(std_classPred)

In [ ]:
std_classPred=std_classPred[["Course_code"	, "Activity_type", "Predicted_class" ]]

In [ ]:
std_classPred.to_csv("./Data/std_classPred.csv" )

In [ ]:
X_test["predicted_Mark"]=y_pred

# Old stuff

## Export_graph_to_HTML

In [ ]:
import plotly
import plotly.graph_objs as go
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')
# Create a trace
data = [go.Scatter(
    x = df['data'],
    y = df['totale_positivi'],
)]
layout = go.Layout(
        xaxis=dict(
            title='Data',    
        ),
        yaxis=dict(
            title='Totale positivi',  
        )
    )
fig = go.Figure(data=data, layout=layout)

In [ ]:
fig.show()

In [ ]:
plotly.offline.plot(fig,filename='positives.html',config={'displayModeBar': False},auto_open = True)

In [ ]:
#webbrowser.open('file://' + os.path.realpath(positives.html))

with open('positives.html', 'r') as f:
    html_string = f.read()

End of Try 1

Try 2

In [ ]:
!pip install plotly
!pip install chart_studio

In [ ]:
import plotly.express as px
import chart_studio

In [ ]:
gapminder = px.data.gapminder()
gapminder

In [ ]:
gapminder = px.data.gapminder()
fig = px.scatter(gapminder.query("year==2007"), x="gdpPercap", y="lifeExp", size="pop", color="continent",
           hover_name="country", log_x=True, size_max=60)
fig.show()

In [ ]:
username = 'Eng.aldoseri' # your username
api_key = '7MU0wS3Gp1VOF6IqRFiw' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [ ]:
import chart_studio.plotly as py
py.plot(fig, filename = 'gdp_per_cap', auto_open=True)

In [ ]:
####### implemented to our dataset

In [ ]:
#gapminder = px.data.gapminder()
fig = px.scatter(dt_st.query("student_ID=='080102434S'"), x="Act_submit_Time", y="Act_mark",  color="Course_code",size="Act_mark",
           hover_name="Act_mark")
fig.show()

In [ ]:
username = 'Eng.aldoseri' # your username
api_key = '7MU0wS3Gp1VOF6IqRFiw' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [ ]:
import chart_studio.plotly as py
py.plot(fig, filename = 'gdp_per_cap', auto_open=True)

End of Try 2

Try 3

In [ ]:
import plotly.graph_objs as go
import plotly as plotly
import plotly.express as px

In [ ]:
data = px.data.gapminder()

data_canada = data[data.country == 'Canada']
fig = px.bar(data_canada, x='year', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
             labels={'pop':'population of Canada'}, height=400)

In [ ]:
plotly.offline.plot(fig, filename= "output_filename" + ".html")


## Graph all the students achievment in a specific subject

In [ ]:
#  Create  the index
# df_avt["Act_submit_Time"]= pd.to_datetime(df_avt.Act_submit_Time)
df_avt.index=df_avt["Act_submit_Time"]
df_avt.sample()

In [ ]:
import plotly.graph_objs as go

@interact(sbj=df_avt.Course_code.unique())
def plot_subject(sbj):
  plt.rcParams.update({'font.size': 25})
  x = np.arange(10)
  layout = go.Layout(
    autosize=False,
    width= 1200 , height= 600 )
  fig=go.Figure(layout=layout)

  fig.add_trace(go.Scatter(      
      x=df_avt[(df_avt.Course_code==sbj)&(
          df_avt.Activity_type=="0")].loc["2020-09":"2021-09", "Act_submit_Time"].sort_index(), 
      y=df_avt[(df_avt.Course_code==sbj)&(
          df_avt.Activity_type=="0")].loc["2020-09":"2021-09", "Act_mark"].sort_index()
      , mode='lines+markers', name="Quiz"))

  fig.add_trace(go.Scatter(
      x=df_avt[(df_avt.Course_code==sbj)&(
          df_avt.Activity_type=="1")].loc["2020-09":"2021-09", "Act_submit_Time"].sort_index(), 
      y=df_avt[(df_avt.Course_code==sbj)&(
          df_avt.Activity_type=="1")].loc["2020-09":"2021-09", "Act_mark"].sort_index()
     , mode='lines+markers' ,  name="Assignment"))



  
  fig.update_layout(hovermode="x" , 
      # title="أداء جميع الطلبة في مادة معينة",
    xaxis_title="Submit Time",
    yaxis_title="Mark",
    # font=dict(import plotly.graph_objs as go

    #     family="Courier New, monospace",
    #     size=18,
    #     color="#7f7f7f")
     title={
        'text': "أداء جميع الطلبة في مادة معينة",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}) 
  fig.show()


## Compare the selected student's Quiz and Assignm. with Average of all students

In [ ]:
df_ActAv=pd.pivot_table(df_avt, index=["Course_code", "Activity_type", "Activity_id"], 
               values=["Act_diff", 	"Act_cp_count", 	"Act_time" , 	"Act_mark"] , aggfunc=("mean")).rename(
   columns={
    "Act_diff": "Act_diff_avg", 	
    "Act_cp_count": "Act_cp_avg", 	
    "Act_time" : "Act_time_avg" , 	
    "Act_mark" : "Act_mark_avg" 
   } 
)

In [ ]:
df_ActAv.head()

In [ ]:
df_ActAv.shape

In [ ]:
df_avt.loc["2020-09"].head()

In [ ]:
df_avt.groupby(["student_ID", "Activity_type"]).agg("count").max()

In [ ]:
df_avt[df_avt.student_ID=="080237875"].loc[: ,["Course_code", "Activity_type" , "Activity_id", "Act_diff", 	"Act_cp_count", 	"Act_time"	,"Act_mark"]]

In [ ]:
df_plt=df_avt[df_avt.student_ID=="100611764"].loc["2020-4":"2021-2" ,["Course_code", "Activity_type" , "Activity_id", "Act_diff", 	"Act_cp_count", 	"Act_time"	,"Act_mark"]]

In [ ]:
df_plt.groupby("Course_code").Activity_id.agg("count")

In [ ]:
df_plt.reset_index(inplace=True)

In [ ]:
df_plt=df_plt.merge( df_ActAv, on=["Activity_type", "Activity_id" ])

In [ ]:
df_plt.keys()

In [ ]:
df_plt.set_index("Act_submit_Time", inplace=True)

In [ ]:
# this is dataset for a particular Student.
df_plt.head()

In [ ]:
df_plt["Activity_type"]=df_plt.Activity_type.astype(int)  

In [ ]:
df_plt[df_plt.Activity_type==0]["Act_mark"]

In [ ]:
df_plt.info()

In [ ]:
df_plt[df_plt.Activity_type==1]["Act_mark"]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import seaborn as sns

In [ ]:
plt.rcParams.update({'font.size': 22})
fig = plt.figure(figsize=(18,8))
ax1 = fig.add_subplot(111)

df_plt[df_plt.Activity_type==0]["Act_mark"].sort_index().plot( marker="o" , color="blue", label="Assignements Mark", ax=ax1)
ax2=ax1.twinx()
df_plt[df_plt.Activity_type==0]["Act_mark_avg"].sort_index().plot( ax=ax2, marker="s" , color="green", label="Average Assignement Marks" )

#plt.legend()
ax1.legend()
ax2.legend(loc="lower right")
ax1.grid()
ax2.set_ylabel("Assign. Avg. Mark")
ax1.set_ylabel("Assign. Mark")




ax1.set(xlabel="Dates",
        #ylabel="Quizzes Marks",
        title="Assign. Mark VS Assign. Avg Mark Timeline"        )
fig.autofmt_xdate(rotation=45)

In [ ]:
#####  TRY TO PLOT IT IN ATRACTIVE WAY

In [ ]:
import plotly.graph_objs as go
@interact(sbj=df_avt.Course_code.unique())
def plot_subject(sbj):
  plt.rcParams.update({'font.size': 25})
  x = np.arange(10)
  layout = go.Layout(
    autosize=False,
    width= 1200 , height= 600 )
  fig=go.Figure(layout=layout)

  fig.add_trace(go.Scatter(      
      x=df_plt[(df_plt.Course_code==sbj)&(
          df_plt.Activity_type=="0")].loc["2020-09":"2021-09", "Act_time"].sort_index(), 
      y=df_plt[(df_plt.Course_code==sbj)&(
          df_plt.Activity_type=="0")].loc["2020-09":"2021-09", "Act_mark"].sort_index()
      , mode='lines+markers', name="Quiz"))



  # fig.add_trace(go.Scatter(      
  #     x=df_plt[(df_plt.Course_code==sbj)&(
  #         df_plt.Activity_type=="1")].loc["2020-09":"2021-09", "Act_submit_Time"].sort_index(), 
  #     y=df_plt[(df_plt.Course_code==sbj)&(
  #         df_plt.Activity_type=="1")].loc["2020-09":"2021-09", "Act_mark"].sort_index()
  #     , mode='lines+markers', name="Assignment"))




  
  fig.update_layout(hovermode="x" , 
      # title="أداء جميع الطلبة في مادة معينة",
    xaxis_title="Submit Time",
    yaxis_title="Mark",
    # font=dict(
    #     family="Courier New, monospace",
    #     size=18,
    #     color="#7f7f7f")
     title={
        'text': "أداء جميع الطلبة في مادة معينة",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}) 
  fig.show()


## Add radio button to the graph

In [ ]:
###### try to add radio button

In [ ]:
std_d={}
for st in  dt_st.student_ID.unique():
  std_d[st]=dt_st[dt_st.student_ID==st]["Course_code"].unique()

In [ ]:
from ipywidgets import interact, Dropdown, Checkbox

stID = Dropdown(options = std_d.keys())
course = Dropdown(options = std_d[stID.value]) # options = std_d[stID.value] is to remove inital error but not that necessary.
box = Checkbox(False, description='Quizz/Assignment')


@interact(st = stID, cs = course, chkbx= box)
def print_cs(st, cs,chkbx ):
    course.options = std_d[st] # Here is the trick, i.e. update course.options based on st, namely stID.value.
    # print(st, cs)
    dt_st.query(f"Course_code=='{cs}' & student_ID=='{st}'")

In [ ]:
std="070327176S"
cs="g5_2"
st_type_0= dt_st.query(f"student_ID=='{std}' & Course_code=='{cs}' & Activity_type=='0'")
st_type_1= dt_st.query(f"student_ID=='{std}' & Course_code=='{cs}' & Activity_type=='1'")




#### to add radio button


def f(actype):
    control = actype
    if control == 'Quiz': 
        data=st_type_0
        #data.plot()
        #plt.show()

    elif control== 'Assignment':
        data=st_type_1
        #data.plot()
        #plt.show() 
    return actype


interact(f, actype = widgets.RadioButtons(
options=['Quiz', 'Assignment'],
description='Choose:',
disabled=False))





############ end of radio button 

dt_st.loc["Act_submit_Time"]=dt_st["Act_submit_Time"].astype(str)
dt_tmp= dt_st.query(f"student_ID=='{std}' & Course_code=='{cs}' ")[["Act_mark", "Activity_id","Act_submit_Time" ]]
dt_tmp["Act_submit_Time"]=pd.to_datetime(dt_tmp.Act_submit_Time, format="%Y-%m-%d")
dt_tmp.sort_values(by= "Act_submit_Time", inplace=True)
# dt_tmp["Act_submit_Time"]= dt_tmp.Act_submit_Time.astype("category")


# selected student activities
dt_tmp

In [ ]:
# All Students having the same Activity as ABove
df_stAct=pd.DataFrame()

avt_type_0= df_avt.query(f" Activity_id=='{std}' & Activity_type=='0'")
avt_type_1= df_avt.query(f"Activity_id=='{std}' & Activity_type=='1'")



Mark_type= avt_type_1   if chk  else  avt_type_0
Time_type= avt_type_1  if chk   else avt_type_0


for col, time in zip(
    dt_tmp["Activity_id"],
    dt_tmp.Act_submit_Time.unique()):
  df_bx=pd.DataFrame()
  df_bx=df_avt.query(f"Activity_id== '{col}' ###### TYPE")[["Activity_id", "Act_mark"]]
  df_bx["Act_submit_Time"]= pd.to_datetime(time).strftime("%Y-%m-%d")
  df_stAct= pd.concat([
                       df_stAct, df_bx
  ])
df_stAct.sort_values(by="Act_submit_Time", inplace=True)
df_stAct["Act_submit_Time"]= df_stAct.Act_submit_Time.astype("category")
df_stAct

In [ ]:
#Combine Graph
import seaborn as sns 
import matplotlib.pyplot as plt

fg, ax= plt.subplots( figsize=(20,8))


sns.boxplot(x="Activity_id" ,  y= "Act_mark", data=df_stAct , ax=ax ,width= 0.3)
sns.lineplot(
    x="Activity_id",
    y= "Act_mark", marker='o',
    data=dt_tmp  , ax=ax )
ax.set_xticklabels(dt_tmp.Act_submit_Time.unique())
plt.title("Student's mark compared to other students" , fontsize=20)
plt.xlabel("Date" , fontsize=20)
plt.ylabel("Mark" , fontsize=20)
plt.xticks(rotation=45)
plt.grid()
plt.show()

